<a href="https://colab.research.google.com/github/pavanraja753/Advanced-Topics-in-Artificial-Intelligence/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Annotate code

Consider a fully connected network constructed using the `__init__` method given below.

Summary of Feed Forward computation:

 $z^{l} = w^{l}a^{l-1} + b^{l} \tag{1}$
 $a^{l} = σ(z_{l}) \tag{2}$ 

We assumed Sigmoid Non Linearity in our example




In [2]:
import numpy as np

class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)  # Number of layers in our neural network including the input layer
        self.sizes = sizes            # Storing the sizes list in object variable to use it other methods in this class
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]  # We need to choose inital parameter numbers to compute the forward step and these weights will be in the gradient descent step
                                      # From the Equation 1, we can see that b_l is a vector and it should be of size number of neurons in the layer l. Another point to note is that index 0 in sizes list 
                                      # represents the input layer and the weights and biases are defined from the hidden layer 1. That is the why we are iterating from index 1 from the sizes list 
        self.weights = [np.random.randn(y, x) # From the Equation 1, it is clear that Weights w should be of size num(previous_layer_neurons) * num(current_layer_neurons). and these weights are initialzed 
                                              # by sampling from the Standard Normal distribution (Mean =0, Varience =1)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):        
        for b, w in zip(self.biases, self.weights): # Iterating over all the layers to compute the feedforward step in neural network in a recursive way. 
            a = sigmoid(np.dot(w, a)+b) # We are applying first the equation 1 : z_l = w_l * a_l-1 + b_l and the Equation 2: a_l = sigmoid(z_l) 
        return a



- Comment every code line in `backprop` with the analytical expression that line evaluates in computing $∇C$

- Schematically apply `backprop` on a network constructed as `net = Network([784, 30, 10])`


# Back Propagation Equations

Summary: The equations of backpropagation

\begin{align}
\delta^L = ∇_{a}C \odot σ ^{'}(z^{L}) \tag{1}
\end{align}

\begin{align}
\delta^l = (W^{l+1})^{T}\delta^{l+1} \odot σ ^{'}(z^{l}) \tag{2} 
\end{align}

Using the error terms we compted the gradients with respect to the Weights and 
Biases in each layer


\begin{align}
\frac{∂C}{∂b^l} = \delta^{l} \tag{3}
\end{align}

\begin{align}
\frac{∂C}{∂w^l} = \delta^{l} (a^{l-1})^{T} \tag{4}
\end{align}

In [19]:
def backprop(self, x, y):
    """Return a tuple "(nabla_b, nabla_w)" representing the
    gradient for the cost function C_x.  "nabla_b" and
    "nabla_w" are layer-by-layer lists of numpy arrays, similar
    to "self.biases" and "self.weights"."""
    nabla_b = [np.zeros(b.shape) for b in self.biases]  # Since gradient is computed for each parameter, we are 
                                                        #initializing the gradients of each bias parameter to be zero 
                                                        #with the same number of entries as biases   
    nabla_w = [np.zeros(w.shape) for w in self.weights] # Since gradient is computed for each parameter, we are 
                                                        #initializing the gradients of each weight parameters to be zero 
                                                        #with the same number of entries as weight
    # feedforward
    activation = x                                      # Activation values "a" for the first layer is equal to input itself 
    activations = [x]                                   # list to store all the activations, layer by layer
                                                        # We need to store all the intermediate activations to compute the backpropgation updates
    zs = []                                             # list to store all the z vectors, layer by layer
                                                        # all the intermediate Linear transformation values are also required for the backpropagation steps

    for b, w in zip(self.biases, self.weights):         # Iterating over all the layers to compute the feedforward step in neural network in a recursive way.         
        z = np.dot(w, activation)+b                     # computation of linear tranformation function z = Wx+b. where x is the activation map from the previous layers
        zs.append(z)                                    # Storing the computations in a list to use it in the backpropagation step. In particualy we need these values to 
                                                        # compuate the derivative of sigmoid at these values. Equation 1 and 2 from the summary of backpropagation equations
        activation = sigmoid(z)                         # Applying sigmoid non-lineariy function f(x) = 1/(1 + exp(-x))
        activations.append(activation)                  # Storing the intermediate activation maps for the bavkpropagation steps. Equation 4 requires these values to compute 
                                                        # the partial derivate of Loss with respect to weights. 
    # backward pass
    delta = (activations[-1] - y) * sigmoid_prime(zs[-1])  # This step computes the derivative of Loss with respect to "z variables" in the last layer of neural network. 
                                                           # Since we assumed to use squared loss function (a_i-y_i)^2 / 2, derivative of Loss with respect to output activation 
                                                           # values is (a_i - y_i) and the derivative of Loss with respect to "z variables" is product of 
                                                           #derivative of Loss with respect to "output activation" * derivative of sigmoid function computed at the z variable 
                                                           # Detailed computation of this step is provided in the below section of this code 
    nabla_b[-1] = delta                                    # Using Equation 3, we are computng the gradient with to "b variables" in the last layer of neural network. 
    nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #Using Equation 4, we are computng the gradient with to "w variables" in the last layer of neural network.
    for l in xrange(2, self.num_layers):               # Now, we are applying the recursive backpropagation step. Since we computed the delta values for the last layer, we use this last layer delta values and 
                                                       # Compute the delta for the previous layers. 
        z = zs[-l]                                     # From Equation 2, inorder to apply compte delta values recursively, we need to computed the derivative of sigmoid at the "z variables"
        sp = sigmoid_prime(z)                          # 3rd term in Equation 2 RHS, requires the derivative of sigmoid at the "z variables"
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp  # Computing the delta values recursively using equation 2 : delta_l = W^(l+1).T * delta_l+1 * derivative of sigma
        nabla_b[-l] = delta                            # Using Equation 3 : C/ partial b = delta, we are computng the gradient with to "b variables" in the current layer of neural network. partial 
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #Using Equation 4: C/ partial W = delta * a.T, we are computng the gradient with to "w variables" in the current layer of neural network.
    return (nabla_b, nabla_w)


- Cost for a single training example is $C_x = \frac{1}{2} ||y-a_L||^2 \tag{1}$

$\frac{∂C}{∂{a_L}} = ||y-a_L||(-1) =||a_L-y|| \tag{2}$

-  From the notation $∇_{a}C = \frac{∂C}{∂{a_L}} = ||a_L-y|| \tag{3}$

The above relation is directly used in the Equation 1 of summary of backpropgation steps.
- This formula will change depending on the choice of loss function

In [79]:
import numpy as np
import random

class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)  # Number of layers in our neural network including the input layer
        self.sizes = sizes            # Storing the sizes list in object variable to use it other methods in this class
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]  # We need to choose inital parameter numbers to compute the forward step and these weights will be in the gradient descent step
                                      # From the Equation 1, we can see that b_l is a vector and it should be of size number of neurons in the layer l. Another point to note is that index 0 in sizes list 
                                      # represents the input layer and the weights and biases are defined from the hidden layer 1. That is the why we are iterating from index 1 from the sizes list 
        self.weights = [np.random.randn(y, x) # From the Equation 1, it is clear that Weights w should be of size num(previous_layer_neurons) * num(current_layer_neurons). and these weights are initialzed 
                                              # by sampling from the Standard Normal distribution (Mean =0, Varience =1)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):        
        for b, w in zip(self.biases, self.weights): # Iterating over all the layers to compute the feedforward step in neural network in a recursive way. 
            a = sigmoid(np.dot(w, a)+b) # We are applying first the equation 1 : z_l = w_l * a_l-1 + b_l and the Equation 2: a_l = sigmoid(z_l) 
        return a


    def backprop(self, x, y):
        """Return a tuple "(nabla_b, nabla_w)" representing the
        gradient for the cost function C_x.  "nabla_b" and
        "nabla_w" are layer-by-layer lists of numpy arrays, similar
        to "self.biases" and "self.weights"."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]  # Since gradient is computed for each parameter, we are 
                                                            #initializing the gradients of each bias parameter to be zero 
                                                            #with the same number of entries as biases   
        nabla_w = [np.zeros(w.shape) for w in self.weights] # Since gradient is computed for each parameter, we are 
                                                            #initializing the gradients of each weight parameters to be zero 
                                                            #with the same number of entries as weight
        # feedforward
        activation = x                                      # Activation values "a" for the first layer is equal to input itself 
        activations = [x]                                   # list to store all the activations, layer by layer
                                                            # We need to store all the intermediate activations to compute the backpropgation updates
        zs = []                                             # list to store all the z vectors, layer by layer
                                                            # all the intermediate Linear transformation values are also required for the backpropagation steps

        for b, w in zip(self.biases, self.weights):         # Iterating over all the layers to compute the feedforward step in neural network in a recursive way.         
            z = np.dot(w, activation)+b                     # computation of linear tranformation function z = Wx+b. where x is the activation map from the previous layers
            zs.append(z)                                    # Storing the computations in a list to use it in the backpropagation step. In particualy we need these values to 
                                                            # compuate the derivative of sigmoid at these values. Equation 1 and 2 from the summary of backpropagation equations
            activation = sigmoid(z)                         # Applying sigmoid non-lineariy function f(x) = 1/(1 + exp(-x))
            activations.append(activation)                  # Storing the intermediate activation maps for the bavkpropagation steps. Equation 4 requires these values to compute 
                                                            # the partial derivate of Loss with respect to weights. 
        # backward pass
        delta = (activations[-1] - y) * sigmoid_prime(zs[-1])  # This step computes the derivative of Loss with respect to "z variables" in the last layer of neural network. 
                                                            # Since we assumed to use squared loss function (a_i-y_i)^2 / 2, derivative of Loss with respect to output activation 
                                                            # values is (a_i - y_i) and the derivative of Loss with respect to "z variables" is product of 
                                                            #derivative of Loss with respect to "output activation" * derivative of sigmoid function computed at the z variable 
                                                            # Detailed computation of this step is provided in the below section of this code 
        nabla_b[-1] = delta                                    # Using Equation 3, we are computng the gradient with to "b variables" in the last layer of neural network. 
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #Using Equation 4, we are computng the gradient with to "w variables" in the last layer of neural network.
        for l in range(2, self.num_layers):               # Now, we are applying the recursive backpropagation step. Since we computed the delta values for the last layer, we use this last layer delta values and 
                                                        # Compute the delta for the previous layers. 
            z = zs[-l]                                     # From Equation 2, inorder to apply compte delta values recursively, we need to computed the derivative of sigmoid at the "z variables"
            sp = sigmoid_prime(z)                          # 3rd term in Equation 2 RHS, requires the derivative of sigmoid at the "z variables"
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp  # Computing the delta values recursively using equation 2 : delta_l = W^(l+1).T * delta_l+1 * derivative of sigma
            nabla_b[-l] = delta                            # Using Equation 3 : C/ partial b = delta, we are computng the gradient with to "b variables" in the current layer of neural network. partial 
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #Using Equation 4: C/ partial W = delta * a.T, we are computng the gradient with to "w variables" in the current layer of neural network.
        return (nabla_b, nabla_w)


    def SGD(self, training_data, epochs, mini_batch_size, eta):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  """
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)


    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))



def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [72]:
net = Network([784,30,10])

In [73]:
net.feedforward(np.random.randn(784,1))

array([[0.82417706],
       [0.00781638],
       [0.98149145],
       [0.98866133],
       [0.55647665],
       [0.91949438],
       [0.00309173],
       [0.43049102],
       [0.0557094 ],
       [0.98392345]])

In [74]:
nabla_b,nabla_w = net.backprop(np.random.randn(784,1),np.array([1,0,0,0,0,0,0,0,0,0]).reshape(10,1))

In [75]:
nabla_b[0].shape, nabla_b[1].shape

((30, 1), (10, 1))

In [76]:
nabla_w[0].shape, nabla_w[1].shape

((30, 784), (10, 30))

In [77]:
training_data = (np.random.randn(10000,784), np.random.randn(10000,10))